In [2]:
import requests
from bs4 import BeautifulSoup, NavigableString
import re
from datetime import datetime
import sqlite3

In [3]:
def insert_data_into_db(data, cursor):
    cursor.execute('''CREATE TABLE IF NOT EXISTS press_releases2 (
                    ID TEXT PRIMARY KEY,
                    TEXT TEXT,
                    TITLE TEXT,
                    PUBDATE TEXT,
                    AUTHOR TEXT,
                    CATEGORIES TEXT,
                    INSTITUTION_ID TEXT,
                    INSTITUTION_NAME TEXT,
                    AUTHOR_JOB TEXT,
                    AUTHOR_NAME TEXT,
                    FIELDS TEXT,
                    RELEVANCE TEXT
                )''')
    """Inserts extracted data into the database."""
    cursor.execute('''INSERT INTO press_releases2 (ID, TEXT, TITLE, PUBDATE, AUTHOR, CATEGORIES, INSTITUTION_ID, INSTITUTION_NAME, AUTHOR_JOB, AUTHOR_NAME, FIELDS, RELEVANCE)
                      VALUES (:ID, :TEXT, :TITLE, :PUBDATE, :AUTHOR, :CATEGORIES, :INSTITUTION_ID, :INSTITUTION_NAME, :AUTHOR_JOB, :AUTHOR_NAME, :FIELDS, :RELEVANCE)''', data)

In [6]:
def extract_data(url, cursor):

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    items = []
    merkmale = soup.find(lambda tag: tag.name == "p" and any("Merkmale dieser Pressemitteilung" in s for s in tag.stripped_strings))
    for content in merkmale.contents:
        if isinstance(content, NavigableString):
            items.append(content.strip())
    items = items[1:]
    
    ul_tag = soup.find('ul', class_='no-bullet')

    for sibling in ul_tag.find_next_siblings():
        if sibling.name == 'p' and not sibling.find('em'):
            text = sibling.text.strip()
            text = re.sub(r'\n', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            break
    
        
    data = {
        "ID": url.split('/')[-1][4:], 
        "TEXT": text,
        "TITLE": soup.find('h4').text.strip(),
        "PUBDATE": datetime.strptime(soup.find('em').text.strip(),'%d.%m.%Y %H:%M').strftime('%Y-%m-%d'),
        "AUTHOR": soup.find('h5', class_='subheader').contents[0].strip(), 
        "CATEGORIES": items[3],
        "INSTITUTION_ID": soup.find('h5', class_='subheader').find('a')['href'].split('/')[-1][11:],
        "INSTITUTION_NAME": soup.find('h5', class_='subheader').find('a').text.strip(),
        "AUTHOR_JOB": soup.find('h5', class_='subheader').find('em').text.strip(),
        "AUTHOR_NAME": soup.find('h5', class_='subheader').contents[0].strip(), 
        "FIELDS": items[1],
        "RELEVANCE": items[2]
    }
    print(data)
    insert_data_into_db(data, cursor)

In [3]:
#extract_data("https://idw-online.de/de/news649090")

In [6]:
start_id = 649336
#end_id = 649350
end_id = 830053

db_name = 'idwdata.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

for news_id in range(start_id, end_id):
    url = f"https://idw-online.de/de/news{news_id}"
    extract_data(url)

conn.commit()
conn.close()
print("Data inserted successfully.")

In [8]:
db_name = 'idwdata.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
extract_data("https://idw-online.de/de/news649090", cursor)
conn.commit()
conn.close()

{'ID': '649090', 'TEXT': 'Wegen der in den vergangenen Monaten drastisch gestiegenen Anzahl von geflüchteten Menschen müssen Kommunen neue Konzepte entwickeln, um unter anderem eine adäquate medizinische Versorgung von Asylsuchenden zu sichern. Zum Bewältigen dieser Herausforderungen hat die Kassenärztliche Vereinigung Sachsen im Herbst 2015 eine Flüchtlingsambulanz in Dresden gegründet. Auch Medizinstudierende der TU Dresden werden seitdem ehrenamtlich in die Arbeit der Flüchtlingsambulanz integriert, jedoch ohne strukturierte Vorbereitung auf die medizinischen und interkulturellen Herausforderungen. Das ändert sich nun durch das neue Wahlpflichtfach „Flüchtlingsversorgung“. „Wir wollen so die fachlichen und kulturellen Kompetenzen unserer Medizin-Studierenden für die Arzt-Patienten-Interaktion und dadurch die künftige medizinische Behandlung von Flüchtlingen verbessern“, erläutert Prof. Antje Bergmann. Die Leiterin des Bereichs Allgemeinmedizin an der Medizinischen Fakultät Carl Gust